# Build a Customer Churn Model for Music Streaming App Users: Model Selection and Model Explainability


## Overview

### What is Customer Churn and why is it important for businesses?

Customer churn, or customer retention/attrition, means a customer has the tendency to leave and stop paying for a business. It is one of the primary metrics companies want to track to get a sense of their customer satisfaction, especially for a subscription-based business model. The company can track churn rate (defined as the percentage of customers churned during a period) as a health indicator for the business, but we would love to identify the at-risk customers before they churn and offer appropriate treatment to keep them with the business, and this is where machine learning comes into play.
### Use Cases for Customer Churn

Any subscription-based business would track customer churn as one of the most critical Key Performance Indicators (KPIs). Such companies and industries include Telecom companies (cable, cell phone, internet, etc.), digital subscriptions of media (news, forums, blogposts platforms, etc.), music and video streaming services, and other Software as a Service (SaaS) providers (e-commerce, CRM, Mar-Tech, cloud computing, video conference provider, and visualization and data science tools, etc.)

### Define Business problem

To start with, here are some common business problems to consider depending on your specific use cases and your focus:

 * Will this customer churn (cancel the plan, cancel the subscription)?
 * Will this customer downgrade a pricing plan?
 * For a subscription business model, will a customer renew his/her subscription?

### Machine learning problem formulation

#### Classification: will this customer churn?

To goal of classification is to identify the at-risk customers and sometimes their unusual behavior, such as: will this customer churn or downgrade their plan? Is there any unusual behavior for a customer? The latter question can be formulated as an anomaly detection problem.

#### Time Series: will this customer churn in the next X months? When will this customer churn?

You can further explore your users by formulating the problem as a time series one and detect when will the customer churn.

### Data Requirements

#### Data collection Sources

Some most common data sources used to construct a data set for churn analysis are:
* Customer Relationship Management platform (CRM), 
* engagement and usage data (analytics services), 
* passive feedback (ratings based on your request), and active feedback (customer support request, feedback on social media and review platforms).

#### Construct a Data Set for Churn Analysis

Most raw data collected from the sources mentioned above are huge and often needs a lot of cleaning and pre-processing. For example, usage data is usually event-based log data and can be more than a few gigabytes every day; you can aggregate the data to user-level daily for further analysis. Feedback and review data are mostly text data, so you would need to clean and pre-process the natural language data to be normalized, machine-readable data. If you are joining multiple data sources (especially from different platforms) together, you would want to make sure all data points are consistent, and the user identity can be matched across different platforms.
           
#### Challenges with Customer Churn

* Business related
    * Importance of domain knowledge: this is critical when you start building features for the machine learning model. It is important to understand the business enough to decide which features would trigger retention.
* Data issues
    * fewer churn data available (imbalanced classes): data for churn analysis is often very imbalanced as most of the customers of a business are happy customers (usually).
    * User identity mapping problem: if you are joining data from different platforms (CRM, email, feedback, mobile app, and website usage data), you would want to make sure user A is recognized as the same user across multiple platforms. There are third-party solutions that help you tackle this problem.
    * Not collecting the right data for the use case or Lacking enough data

## Model Selection

You can experiment with all your model choices and see which one gives better results. A few things to note when you choose algorithms:
* **Start with simple ones**: Usually for tabular data classification that does not contain complex unstructured data (text, audio, image, etc.), you can start with logistic regression to see how your data performs, as sometimes the simplest model gives great results if your data have a strong linear pattern.

* **Think about your data structure**: For imbalanced class data like churn analysis, you can experiment with tree-based models like the random forest, gradient boosting, or XGboost since they are less sensitive to class imbalance.

* **Interpretability**: logistic regression model generally has better interpretability because of its linearity. You can also use feature importance from tree-based models or Support Vector Machines as an overall observation, but not to your predicting instance level. Instead, you can utilize tools like SHAP or the SageMaker new feature SageMaker Clarify to better visualize which feature contributing more to your prediction results.

In this use case, a tree-based model XGBoost is chosen due to consideration of imbalanced class, and in the family of tree based models, XGBoost usually gives best results as its built for model performance and computational speed. 

## Training with SageMaker Estimator and Experiment

Once you decide on a range of models you want to experiment with, you can start training and comparing model results to choose the best one. A few things left for you to make a decision:
* SageMaker estimator configuration
  * to initialize your training job, you would need to config your SageMaker estimator and SageMaker training image by specifying the model choice, instance size, and type.
* Choose evaluation methods
    * You can check the [model parameter documentation page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#learning-task-parameters) for all the evaluation metrics you can choose for a model. For a imbalanced classification problem, you can choose F1 as your evaluation especially for comparing different models;  area under curve (auc) is also a good choice when your output is probability.
* Hyper-parameters
    * You can look at the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) for a complete list of hyper-parameters tunable for the model (The XGBoost model here was given as an example). For best performances, you can experiment with a range of combinations for the hyper-parameters and compare the validation results.
   
### How to create a training job as a trial in SageMaker Experiment    

#### Get ECR image URIs for pre-built SageMaker Docker images

In [ ]:
! pip install sagemaker-experiments

In [ ]:
import sagemaker
import json
import pandas as pd
import glob
import s3fs
import boto3
from datetime import datetime
import os

In [3]:
sagemaker_session = sagemaker.Session()
s3 = sagemaker_session.boto_session.resource("s3")

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
smclient = boto3.Session().client("sagemaker")
bucket = sagemaker_session.default_bucket()
prefix = "music-streaming"

<a id='4'></a>

### Download Data and Upload to S3

We ingest the simulated data from the public SageMaker S3 training database. If you want to see how the train, test, and validation datasets are created in detail, look at [Build a Customer Churn Model for Music Streaming App Users: Overview and Data Preparation](0_cust_churn_overview_dw.ipynb)

In [4]:
##### Alternative: copy data from a public S3 bucket to your own bucket
##### data file should include full_data.csv and sample.json
#### cell 5 - 7 is not needed; the processing job before data wrangler screenshots is not needed
!mkdir -p data/raw
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    "datasets/tabular/customer-churn/customer-churn-data-v2.zip",
    "data/raw/customer-churn-data.zip",
)

In [ ]:
!unzip -o ./data/raw/customer-churn-data.zip -d ./data

In [ ]:
# unzip the partitioned data files into the same folder
!unzip -o data/simu-1.zip -d data/raw
!unzip -o data/simu-2.zip -d data/raw
!unzip -o data/simu-3.zip -d data/raw
!unzip -o data/simu-4.zip -d data/raw

In [7]:
!rm ./data/raw/*.zip

In [ ]:
!unzip -o data/sample.zip -d data/raw

In [ ]:
!aws s3 cp  ./data/raw s3://$bucket/$prefix/data/json/  --recursive

In [10]:
s3_input_train = (
    boto3.Session()
    .resource("s3")
    .Bucket(bucket)
    .Object(os.path.join(prefix, "train/train.csv"))
    .upload_file("data/train_updated.csv")
)
s3_input_validation = (
    boto3.Session()
    .resource("s3")
    .Bucket(bucket)
    .Object(os.path.join(prefix, "validation/validation.csv"))
    .upload_file("data/validation_updated.csv")
)
s3_input_validation = (
    boto3.Session()
    .resource("s3")
    .Bucket(bucket)
    .Object(os.path.join(prefix, "test/test_labeled.csv"))
    .upload_file("data/test_updated.csv")
)

#### Initialize Model Hyperparameters

In [11]:
hyperparameters = {
    "max_depth": "12",
    "eta": "0.08",
    "gamma": "4",
    "min_child_weight": "7",
    "subsample": "0.7",
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "num_round": "800",
    "early_stopping_rounds": "50",
}

#### Define SageMaker estimator

In [ ]:
%%time
from time import gmtime, strftime

container = sagemaker.image_uris.retrieve(
    "xgboost", region, version="1.0-1", instance_type="ml.m4.xlarge"
)


xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
)

xgb.set_hyperparameters(**hyperparameters)

In [13]:
from sagemaker.inputs import TrainingInput

content_type = "csv"
train_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/{}/".format(bucket, prefix, "validation"), content_type=content_type
)

In [ ]:
%%time
xgb.fit(inputs={"train": train_input, "validation": validation_input}, wait=True)

#### Define SageMaker Experiment and Trial

In [15]:
# custom trial name
experiment_name = "music-streaming-churn-exp-{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))
trial_name_xgb = "xgboost-{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
%%time
from smexperiments import experiment, trial
from sagemaker import analytics

# create experiment if it doesn't exist
try:
    my_experiment = experiment.Experiment.load(experiment_name=experiment_name)
    print(f"Experiment loaded {experiment_name}: SUCCESS")
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        my_experiment = experiment.Experiment.create(experiment_name=experiment_name)
        print(f"Experiment creation {experiment_name}: SUCCESS")

# create the trial if it doesn't exist
try:
    my_trial = trial.Trial.load(trial_name=trial_name_xgb)
    print(f"Trial loaded {trial_name_xgb}: SUCCESS")
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        my_trial = trial.Trial.create(experiment_name=experiment_name, trial_name=trial_name_xgb)
        print(f"Create trial {my_trial.trial_name}: SUCCESSFUL")


xgb.fit(
    inputs={"train": train_input, "validation": validation_input},
    wait=True,
    experiment_config={
        "ExperimentName": my_experiment.experiment_name,
        "TrialName": my_trial.trial_name,
        "TrialComponentDisplayName": "churn-xgboost",
    },
    logs=True,
)

trial_component_analytics = analytics.ExperimentAnalytics(
    experiment_name=my_experiment.experiment_name
)

analytic_table = trial_component_analytics.dataframe()
analytic_table

## Hyperparameter Tuning with SageMaker Hyperparameter Tuning Job

Now that you understand how training one model works and how to create a SageMaker experiment, and selected the XGBoost model as the final model, you will need to fine-tune the hyperparameters for the best model performances. For a xgboost model, you can start with defining ranges for the eta, alpha, min_child_weight, and max_depth. You can check the [documentation when considering what haperparameter to tune](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-considerations.html).

### Specify the Hyperparameter Tuning Job Settings

To specify settings for the hyperparameter tuning job, you define a JSON object. You pass the object as the value of the HyperParameterTuningJobConfig parameter to CreateHyperParameterTuningJob when you create the tuning job.

In [17]:
tuning_job_config = {
    "ParameterRanges": {
        "CategoricalParameterRanges": [],
        "ContinuousParameterRanges": [
            {"MaxValue": "1", "MinValue": "0", "Name": "eta"},
            {"MaxValue": "2", "MinValue": "0", "Name": "alpha"},
            {"MaxValue": "10", "MinValue": "1", "Name": "min_child_weight"},
        ],
        "IntegerParameterRanges": [{"MaxValue": "10", "MinValue": "1", "Name": "max_depth"}],
    },
    "ResourceLimits": {"MaxNumberOfTrainingJobs": 20, "MaxParallelTrainingJobs": 3},
    "Strategy": "Bayesian",
    "TrainingJobEarlyStoppingType": "Auto",
    "HyperParameterTuningJobObjective": {"MetricName": "validation:auc", "Type": "Maximize"},
}

### Configure the Training Jobs

To configure the training jobs that the tuning job launches, define a JSON object that you pass as the value of the TrainingJobDefinition parameter of the CreateHyperParameterTuningJob call.

In [18]:
s3_input_train = "s3://{}/{}/train".format(bucket, prefix)
s3_input_validation = "s3://{}/{}/validation".format(bucket, prefix)

training_job_definition = {
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_train,
                }
            },
        },
        {
            "ChannelName": "validation",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_validation,
                }
            },
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)},
    "ResourceConfig": {"InstanceCount": 2, "InstanceType": "ml.c4.2xlarge", "VolumeSizeInGB": 10},
    "RoleArn": role,
    "StaticHyperParameters": {
        "eval_metric": "auc",
        "num_round": "100",
        "objective": "binary:logistic",
        "rate_drop": "0.3",
        "tweedie_variance_power": "1.4",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 43200},
}

### Name and Launch the Hyperparameter Tuning Job

Now you can provide a name for the hyperparameter tuning job and then launch it by calling the CreateHyperParameterTuningJob API. Pass tuning_job_config, and training_job_definition that you created in previous steps as the values of the parameters.

In [19]:
# custom a tuner job name
tuning_job_name = "ChurnPredictTune-{}".format(datetime.now().strftime("%Y%m%d-%H%M%S"))

In [20]:
# check if tuner job has been created
list_tuning_job = smclient.list_hyper_parameter_tuning_jobs(NameContains=tuning_job_name)
job_results = [[i for i in list_tuning_job[x]] for x in list_tuning_job.keys()]

In [ ]:
smclient.list_hyper_parameter_tuning_jobs(NameContains=tuning_job_name)

In [ ]:
from sagemaker.tuner import HyperparameterTuner

# create the tuning job if it doesn't exist
try:
    if tuning_job_name == job_results[0][0]["HyperParameterTuningJobName"]:
        print(f"Tuning job exists")
except Exception as ex:
    # create hyperparameter tuning job
    smclient.create_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuning_job_name,
        HyperParameterTuningJobConfig=tuning_job_config,
        TrainingJobDefinition=training_job_definition,
    )
    print(f"Create tuning job {tuning_job_name}: SUCCESSFUL")

### Monitor the Progress of a Hyperparameter Tuning Job

To monitor the progress of a hyperparameter tuning job and the training jobs that it launches, you can use the Amazon SageMaker console.

<div>
<img src="image/hp1.PNG" width="600"/>
   </div>
   


## Deploy the model with SageMaker Batch-transform

You can directly deploy the best model from your hyperparameter tuning job by getting the best training job from your tuner.

In [ ]:
%%time
# check status
import time

status = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)["HyperParameterTuningJobStatus"]

while status == "InProgress":
    print(status)
    time.sleep(60)
    status = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuning_job_name
    )["HyperParameterTuningJobStatus"]

print(status)

In [ ]:
%%time
# Attach to an existing hyperparameter tuning job.
tuning_job_details = smclient.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)
xgb_tuner = HyperparameterTuner.attach(
    tuning_job_name,
    job_details=tuning_job_details,
    sagemaker_session=sagemaker_session,
    estimator_cls=None,
)

# Get the best XGBoost training job name from the HPO job
xgb_best_training_job = xgb_tuner.best_training_job()
print(xgb_best_training_job)
# Attach estimator to the best training job name
best_estimator = sagemaker.estimator.Estimator.attach(xgb_best_training_job)

# Create model to be passed to the inference pipeline
best_model = sagemaker.model.Model(
    model_data=best_estimator.model_data,
    role=sagemaker.get_execution_role(),
    image_uri=best_estimator.image_uri,
)

predictor = best_model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

You can also use the following code to find and deploy the best model. Replace with your best model output path. Go to SageMaker Console $\rightarrow$ Hyperparameter Tuning Job $\rightarrow$ [Your hyperparameter Tuning Job] $\rightarrow$ Best model $\rightarrow$ Output. You can also choose to **create a model** from the console under **best training job**.

In [25]:
# replace with your best model output path
# model_artifacts = 's3://{}/{}/output/ChurnPrediction-TuningJob-020-20eee831/output/model.tar.gz'.format(bucket, prefix)
# best_model = sagemaker.model.Model(
#    model_data= model_artifacts,
#    image_uri =container,
#    role=role)

In [26]:
import os

# The location of the test dataset
test_data = pd.read_csv("data/test_w_header.csv")
test_set = test_data.drop(columns=["user_churned"])
test_set.to_csv("data/test.csv", index=False, header=False)
s3_input_validation = (
    boto3.Session()
    .resource("s3")
    .Bucket(bucket)
    .Object(os.path.join(prefix, "test/testdata/test.csv"))
    .upload_file("data/test.csv")
)

batch_input = "s3://{}/{}/test/testdata".format(bucket, prefix)

batch_output = "s3://{}/{}/batch-inference".format(bucket, prefix)

In [ ]:
transformer = best_model.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_output
)
transformer.transform(
    data=batch_input, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

Wait for the previous step to be completed. Once done, you can download the prediction results to your instance for further analysis.

In [28]:
s3.download_file(bucket, prefix + "/batch-inference/test.csv.out", "batch_results")

In [29]:
with open("batch_results") as f:
    results = f.readlines()

## Compare model results to actual

You can evaluate your model results with the test data you left out earlier and check the precision and recall. In customer churn problem, precision and recall means:

* Precision – Of all the users that the algorithm predicts will churn, how many of them do actually churn?
* Recall – What percentage of users that end up churning does the algorithm successfully find?

In [ ]:
test_data["predicted_results"] = pd.to_numeric(results)
# define a threshold to convert probability to class, you can set as 0.5 by default
test_data["predicted_binary"] = [1 if x >= 0.5 else 0 for x in test_data["predicted_results"]]
test_data[["user_churned", "predicted_results", "predicted_binary"]]

You did a good job and your model can detect 85% of the users who are going to churn. 

In [ ]:
from sklearn import metrics

test_labels = test_data["user_churned"]
test_pred = test_data["predicted_binary"]

test_f1 = metrics.f1_score(test_labels, test_pred, average=None)
# fbeta_test= metrics.f1_score(mtest_labels, mpreds_test_xgb, average=None)
prec, rec, fbeta_test, support = metrics.precision_recall_fscore_support(
    test_labels, test_pred, average=None
)
metrics.precision_recall_fscore_support(test_labels, test_pred, average=None)
print("Test Evaluation: ")
print("Average F1 Score: ", (test_f1[0] + test_f1[1]) / 2)
print("Precision Score: ", (prec[1]))
print("Recall Score: ", (rec[1]))